In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
df

C:\Users\akmal\AppData\Local\Temp\ipykernel_15624\513246612.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [4]:
N_FEATURES = len(df.columns)

In [5]:
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [6]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [7]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(N_FEATURES)
    ])

d:\UB\Dicoding\GitHub\Dicoding_Akmal\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True
 
callbacks = myCallback()

In [10]:
# Kode untuk melakukan menyusun struktur sesuai dengan machine learning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) 
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

In [12]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=10,
          callbacks=callbacks,
          verbose=1
    )

Epoch 1/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.0589 - mae: 0.0589 - val_loss: 0.0593 - val_mae: 0.0593
Epoch 2/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.0581 - mae: 0.0581 - val_loss: 0.0570 - val_mae: 0.0570
Epoch 3/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.0578 - mae: 0.0578 - val_loss: 0.0566 - val_mae: 0.0566
Epoch 4/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0561 - val_mae: 0.0561
Epoch 5/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.0571 - mae: 0.0571 - val_loss: 0.0562 - val_mae: 0.0562
Epoch 6/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.0573 - mae: 0.0573 - val_loss: 0.0574 - val_mae: 0.0574
Epoch 7/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.0571 - mae: 0.0571 - val_loss: 0.0561 - val_mae: 0.0561
Epoch 8/10
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.0566 - mae: 0.0566 - val_loss: 0.0563 - val_mae: 0.0563
Epoch 9/10
1349/1349 ━━━━━

In [15]:
from numpy import array, float32


train_pred = model.predict(train_set)
train_pred[0][0]
# Output
array([ 3.8611603e-01,  2.5834844e-01,  2.5597256e-02,  3.8636056e-01,
       -4.1037248e-04,  1.7979867e-03,  8.6604381e-01], dtype=float32)

      1/Unknown 0s 34ms/step

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step


array([ 3.8611603e-01,  2.5834844e-01,  2.5597256e-02,  3.8636056e-01,
       -4.1037248e-04,  1.7979867e-03,  8.6604381e-01], dtype=float32)